In [1]:
import pandas as pd

data = pd.read_json(r'..\..\..\data\all_trees.json')

In [2]:
def str_2_value(string) :
  try:
    integer = float(string)
  except ValueError:
    integer = 0

  return integer

In [3]:
# 所有植物数据
plant_values = []

for i in range(len(data)):
  # 存放单株植物数据
  plant_value = []
  
  # 习性
  habit = data.loc[i]['template']['habit']
  plant_value.append(habit)
  
  # 三维
  height = data.loc[i]["height"]
  radius = data.loc[i]["radius"]
  subHeight = data.loc[i]["subHeight"]
  ##字符串转换为数值
  height = str_2_value(height)
  radius = str_2_value(radius)
  subHeight = str_2_value(subHeight)
  plant_value.append(height)
  plant_value.append(radius)
  plant_value.append(subHeight)

  #认养情况
  if(data.loc[i,['owner']].isnull().values.any() == False) :
    owner = "已被认养"
  else :
    owner = "未被认养"
  plant_value.append(owner)

  # 放入数组
  if(height == 0 or radius == 0 or subHeight == 0) :
    continue
  else :
    plant_values.append(plant_value)

In [7]:
from pyecharts import options as opts
from pyecharts.charts import Parallel


c = (Parallel()
     .add_schema(
      [
        opts.ParallelAxisOpts(
          dim = 0,
          name = "植物习性",
          type_ = "category",
          data = ["草本", "灌木", "乔木", "藤本"],
        ),
        opts.ParallelAxisOpts(
          dim = 1, 
          name = "植物树高（米）",
          is_trigger_event = True,
        ),
        opts.ParallelAxisOpts(
          dim = 2, 
          name = "植物胸径（厘米）",
          max_ = 210),
        opts.ParallelAxisOpts(
          dim = 3, 
          name = "植物冠幅（米）",
          type_= "log",
          log_base = 15,
          max_ = 64),
        opts.ParallelAxisOpts(
          dim = 4,
          name = "认养情况",
          type_ = "category",
          data = ["已被认养", "未被认养"],
        ),
      ]
    )
    .add(
      "平行坐标轴", 
      plant_values,
      is_smooth = True,
      linestyle_opts = opts.LineStyleOpts(
        width = 1,
        opacity = 1,
        type_= "solid"),
      itemstyle_opts = opts.ItemStyleOpts(
        border_color="black"
      ),
      tooltip_opts = opts.TooltipOpts(
        is_show = True,
        border_width = 1,
        formatter = "{c}"
      ))
    .set_global_opts(
      title_opts = opts.TitleOpts(
        #title = "植物三维"
      ),
      visualmap_opts = opts.VisualMapOpts(
        is_show = True,
        min_ = 0,
        max_ = 35,
        dimension = 1,
        range_color = [
          "#80b1d3", #蓝色
          "#bebada", #紫色
          "#FFE65F", #黄色
          "#b3de69", #绿色
          "#8dd3c7", #蓝绿色
        ]
      ),
    )
)

c.render("parallel.html")
c.render_notebook()
